# Lecture Plan

1. Vertex AI Training
2. Vertex AI Testing

---

In [1]:
# authenticate the user
from google.colab import auth
auth.authenticate_user()

In [ ]:
! pip3 install google-cloud-aiplatform google-cloud-storage

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import google.cloud.aiplatform
# if you get an error just restart the runtime and run again

import time

In [4]:
# Define Project ID
PROJECT_ID = "theta-cell-406519"

! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [5]:
# Define the region
REGION = "us-central1"


In [ ]:
# Define the bucket
BUCKET_NAME = f"project-2-trng-1855"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
# resolve an import issue related to shapely
! pip install -U google-cloud-aiplatform "shapely<2"


In [6]:
from google.cloud import aiplatform, storage
from google.cloud.aiplatform import jobs

In [7]:
aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
# Creating the training Dataset
src_uris = "gs://project-2-trng-1855/training-data.csv"
display_name = "trng-dataset-ip"

text_dataset = aiplatform.TextDataset.create(
    display_name=display_name,
    gcs_source=src_uris,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
    sync=True,
)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TextDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TextDataset backing LRO: projects/719559140092/locations/us-central1/datasets/1494268187984789504/operations/3235994593220624384
INFO:google.cloud.aiplatform.datasets.dataset:TextDataset created. Resource name: projects/719559140092/locations/us-central1/datasets/1494268187984789504
INFO:google.cloud.aiplatform.datasets.dataset:To use this TextDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TextDataset('projects/719559140092/locations/us-central1/datasets/1494268187984789504')
INFO:google.cloud.aiplatform.datasets.dataset:Importing TextDataset data: projects/719559140092/locations/us-central1/datasets/1494268187984789504
INFO:google.cloud.aiplatform.datasets.dataset:Import TextDataset data backing LRO: projects/719559140092/locations/us-central1/datasets/1494268187984789504/operations/8260885877459255296
INFO:google.cloud.aipl

In [ ]:
# Define the training job
training_job_display_name = "training-job-unique"
job = aiplatform.AutoMLTextTrainingJob(
    display_name=training_job_display_name,
    prediction_type="classification",
    multi_label=False,
)

In [ ]:
model_display_name = "ip-classification"

# Run the training job
model = job.run(
    dataset=text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.5,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    sync=True,
)

INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2746537646267301888?project=719559140092
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/719559140092/locations/us-central1/trainingPipelines/2746537646267301888 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/719559140092/locations/us-central1/trainingPipelines/2746537646267301888 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/719559140092/locations/us-central1/trainingPipelines/2746537646267301888 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/719559140092/locations/us-central1/trainingPipelines/2746537646267301888 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.

In [13]:
# Evaluation Metrics

model = aiplatform.ModelRegistry('4831586144959332352')

model = model.get_model()

model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

{'name': 'projects/719559140092/locations/us-central1/models/4831586144959332352@1/evaluations/8584133173514862592', 'metricsSchemaUri': 'gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml', 'metrics': {'logLoss': 0.12222175, 'confidenceMetrics': [{'recall': 1.0, 'precision': 0.33333334, 'recallAt1': 0.9166667, 'confidenceThreshold': 0.0, 'f1Score': 0.5, 'maxPredictions': 0.0, 'precisionAt1': 0.9166667, 'f1ScoreAt1': 0.9166667}, {'f1ScoreAt1': 0.9166667, 'confidenceThreshold': 0.05, 'precisionAt1': 0.9166667, 'maxPredictions': 0.0, 'f1Score': 0.75, 'precision': 0.6, 'recall': 1.0, 'recallAt1': 0.9166667}, {'f1ScoreAt1': 0.9166667, 'f1Score': 0.82758623, 'precisionAt1': 0.9166667, 'confidenceThreshold': 0.1, 'recall': 1.0, 'recallAt1': 0.9166667, 'precision': 0.7058824, 'maxPredictions': 0.0}, {'recallAt1': 0.9166667, 'maxPredictions': 0.0, 'precision': 0.85714287, 'f1ScoreAt1': 0.9166667, 'confidenceThreshold': 0.15, 'precisionAt1': 0.9166667, 'f1Scor

[evaluation metrics]('https://cloud.google.com/vertex-ai/docs/evaluation/introduction#text')

In [ ]:
# Model Deployment
deployed_model_display_name = "trng-1855-ip-classification"

endpoint = model.deploy(
    deployed_model_display_name=deployed_model_display_name, sync=True)

In [ ]:
# Model Prediction

content = "I'm having trouble accessing certain websites with my internet connection."

response = endpoint.predict(instances=[{"content": content}])

In [17]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [START aiplatform_predict_text_classification_single_label_sample]
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_text_classification_single_label_sample(
    project: str,
    endpoint_id: str,
    content: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    instance = predict.instance.TextClassificationPredictionInstance(
        content=content,
    ).to_value()
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)

    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))


# [END aiplatform_predict_text_classification_single_label_sample]

In [18]:
#Model prediction for a deployed model
model = aiplatform.ModelRegistry('4831586144959332352')

model = model.get_model()
predict_text_classification_single_label_sample(
    project="719559140092",
    endpoint_id="8395934561371947008",
    location="us-central1",
    content="I'm having trouble accessing certain websites with my internet connection."
)

response
 deployed_model_id: 4310637535719063552
 prediction: {'confidences': [0.9815472364425659, 0.003638351336121559, 0.014814476482570171], 'ids': ['5433577832137621504', '8315881593654738944', '3127734822923927552'], 'displayNames': ['Service', 'Sales', 'General']}


In [1]:
# code for online prediction

def predict_text_classification_single_label_sample(
    project, location, endpoint, content
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    response = endpoint.predict(instances=[{"content": content}], parameters={})

    for prediction_ in response.predictions:
        print(prediction_)

[Model bias and fairness](https://cloud.google.com/vertex-ai/docs/evaluation/model-bias-metrics)